In [1]:
from IPython import get_ipython


def type_of_script():
    """
    Detects and returns the type of python kernel
    :return: string 'jupyter' or 'ipython' or 'terminal'
    """
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return 'jupyter'
        if 'terminal' in ipy_str:
            return 'ipython'
    except:
        return 'terminal'


if type_of_script() == 'jupyter':
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
    os.nice(19)
    
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize
import matplotlib.gridspec as gridspec

import numpy as np
import os, glob, atexit, signal
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=0
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
from importlib import reload
from copy import deepcopy
from multiprocessing import Pool, shared_memory, Value

from common_func import *

In [2]:
mode1 = 'nh'
database1 = f"/home/jeon/MissingSat/database/{mode1}"
iout1 = 1026
repo1, rurmode1, dp1 = mode2repo(mode1)
snap1 = uri.RamsesSnapshot(repo1, iout1, mode=rurmode1)
snap1s = uri.TimeSeries(snap1)
snap1s.read_iout_avail()
nout1 = snap1s.iout_avail['iout']; nout=nout1[nout1 <= iout1]
gal1s = uhmi.HaloMaker.load(snap1, galaxy=True, double_precision=dp1)
hal1s = uhmi.HaloMaker.load(snap1, galaxy=False, double_precision=dp1)

mode2 = 'nh2'
database2 = f"/home/jeon/MissingSat/database/{mode2}"
iout2 = 797
repo2, rurmode2, dp2 = mode2repo(mode2)
snap2 = uri.RamsesSnapshot(repo2, iout2, mode=rurmode2)
snap2s = uri.TimeSeries(snap2)
snap2s.read_iout_avail()
nout2 = snap2s.iout_avail['iout']; nout=nout2[nout2 <= iout2]
gal2s = uhmi.HaloMaker.load(snap2, galaxy=True, double_precision=dp2)
hal2s = uhmi.HaloMaker.load(snap2, galaxy=False, double_precision=dp2)



LG1 = pklload(f"{database1}/LocalGroup.pickle")
allsubs1 = None
states1 = None
for key in LG1.keys():
    subs = LG1[key]['subs']
    real = LG1[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True
    subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'
    state[~ind] = 'pair'
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsubs1 = subs if allsubs1 is None else np.hstack((allsubs1, subs))
    states1 = state if states1 is None else np.hstack((states1, state))
argsort = np.argsort(allsubs1['id'])
allsubs1 = allsubs1[argsort]
states1 = states1[argsort]

LG2 = pklload(f"{database2}/LocalGroup.pickle")
allsubs2 = None
states2 = None
for key in LG2.keys():
    subs = LG2[key]['subs']
    real = LG2[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True
    subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'
    state[~ind] = 'pair'
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsubs2 = subs if allsubs2 is None else np.hstack((allsubs2, subs))
    states2 = state if states2 is None else np.hstack((states2, state))
argsort = np.argsort(allsubs2['id'])
allsubs2 = allsubs2[argsort]
states2 = states2[argsort]

# NH

In [3]:
stree1 = pklload(f"{database1}/stable_tree.pickle")

In [31]:
fnames = os.listdir(f"{database1}/stable_prog/props")
fnames = [fname for fname in fnames if(fname.startswith("insitu"))]
fnames.sort()
insitus = {}
for fname in tqdm(fnames):
    tmp = pklload(f"{database1}/stable_prog/props/{fname}")
    iout = int(fname.split('.')[0][-5:])
    insitus[iout] = tmp

  0%|          | 0/848 [00:00<?, ?it/s]

In [34]:
keys = list(stree1.keys())
ntree1 = {}
for i, key in tqdm(enumerate(keys), total=len(keys)):
    ibranch = stree1[key]
    idtype = ibranch.dtype
    ndtype = idtype.descr + [('insitu', '<f8'), ('insitu_vir', '<f8')]
    nbranch = np.zeros(len(ibranch), dtype=ndtype)
    for iname in idtype.names:
        nbranch[iname] = ibranch[iname]
    for j, iout in enumerate(nbranch['timestep']):
        tmp = insitus[iout][i]
        nbranch[j]['insitu'] = tmp['insitu']
        nbranch[j]['insitu_vir'] = tmp['insitu_vir']
    ntree1[key] = nbranch

  0%|          | 0/1336 [00:00<?, ?it/s]

In [40]:
# change 'stable_tree.pickle' to 'stable_tree.pickle.old'
os.rename(f"{database1}/stable_tree.pickle", f"{database1}/stable_tree.pickle.old")
# save new 'stable_tree.pickle'
pklsave(ntree1, f"{database1}/stable_tree.pickle")

# NH2

In [41]:
stree2 = pklload(f"{database2}/stable_tree.pickle")

In [42]:
fnames = os.listdir(f"{database2}/stable_prog/props")
fnames = [fname for fname in fnames if(fname.startswith("insitu"))]
fnames.sort()
insitus = {}
for fname in tqdm(fnames):
    tmp = pklload(f"{database2}/stable_prog/props/{fname}")
    iout = int(fname.split('.')[0][-5:])
    insitus[iout] = tmp

  0%|          | 0/781 [00:00<?, ?it/s]

In [43]:
keys = list(stree2.keys())
ntree2 = {}
for i, key in tqdm(enumerate(keys), total=len(keys)):
    ibranch = stree2[key]
    idtype = ibranch.dtype
    ndtype = idtype.descr + [('insitu', '<f8'), ('insitu_vir', '<f8')]
    nbranch = np.zeros(len(ibranch), dtype=ndtype)
    for iname in idtype.names:
        nbranch[iname] = ibranch[iname]
    for j, iout in enumerate(nbranch['timestep']):
        tmp = insitus[iout][i]
        nbranch[j]['insitu'] = tmp['insitu']
        nbranch[j]['insitu_vir'] = tmp['insitu_vir']
    ntree2[key] = nbranch

  0%|          | 0/1242 [00:00<?, ?it/s]

In [44]:
# change 'stable_tree.pickle' to 'stable_tree.pickle.old'
os.rename(f"{database2}/stable_tree.pickle", f"{database2}/stable_tree.pickle.old")
# save new 'stable_tree.pickle'
pklsave(ntree2, f"{database2}/stable_tree.pickle")